<a href="https://colab.research.google.com/github/DavidCastroPena/credit-models/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
# Paths to your files
import pandas as pd

# First attempt with folder ID
path1_10_present = '/content/drive/My Drive/11S7cbE_9RF5lE2TuDVrIgcdrRnwJjwTY/10_present.csv'
path1_91_09 = '/content/drive/My Drive/11S7cbE_9RF5lE2TuDVrIgcdrRnwJjwTY/91_09.csv'

# Second attempt with folder name
path2_10_present = '/content/drive/My Drive/MS&E246/10_present.csv'
path2_91_09 = '/content/drive/My Drive/MS&E246/91_09.csv'

# Try reading with both paths
try:
    # Try first path
    df_10_present = pd.read_csv(path1_10_present)
    df_91_09 = pd.read_csv(path1_91_09)
    print("Successfully read from folder ID path")
except:
    try:
        # Try second path
        df_10_present = pd.read_csv(path2_10_present)
        df_91_09 = pd.read_csv(path2_91_09)
        print("Successfully read from folder name path")
    except Exception as e:
        print(f"Error: {e}")

<ipython-input-6-4238d0bcc21f>:22: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_10_present = pd.read_csv(path2_10_present)


Successfully read from folder name path


<ipython-input-6-4238d0bcc21f>:23: DtypeWarning: Columns (12,13,14,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_91_09 = pd.read_csv(path2_91_09)


In [ ]:
#What is the id of the dataset
id_columns = ['AsOfDate', 'Program', 'BorrName', 'BorrStreet', 'BorrCity', 'BorrState', 'BorrZip']

# Check for 10_present dataset
duplicates_10 = df_10_present.groupby(id_columns).size().reset_index(name='count')
has_duplicates_10 = any(duplicates_10['count'] > 1)

# Check for 91_09 dataset
duplicates_91 = df_91_09.groupby(id_columns).size().reset_index(name='count')
has_duplicates_91 = any(duplicates_91['count'] > 1)

print("Duplicates in 10_present:", has_duplicates_10)
print("Duplicates in 91_09:", has_duplicates_91)

# If there are duplicates, let's see some examples
if has_duplicates_10:
    print("\nExample duplicates in 10_present:")
    print(duplicates_10[duplicates_10['count'] > 1].head())

if has_duplicates_91:
    print("\nExample duplicates in 91_09:")
    print(duplicates_91[duplicates_91['count'] > 1].head())

Duplicates in 10_present: True
Duplicates in 91_09: True

Example duplicates in 10_present:
      AsOfDate  Program                   BorrName                BorrStreet  \
80   9/30/2024      504       1013 Enterprises LLC      1013 Lincoln Highway   
150  9/30/2024      504  10820 Hemlock Avenue, LLC      10820 Hemlock Avenue   
195  9/30/2024      504    1128 Petro Parkway, LLC    1128 Petroleum Parkway   
308  9/30/2024      504           123 Western, LLC  123 North Western Avenue   
528  9/30/2024      504                  1508, LLC  1508 Kaliste Saloom Road   

         BorrCity BorrState  BorrZip  count  
80   Schererville        IN    46375      2  
150       Fontana        CA    92337      2  
195     Broussard        LA    70518      2  
308       Chicago        IL    60612      2  
528     Lafayette        LA    70508      2  

Example duplicates in 91_09:
      AsOfDate  Program                        BorrName  \
236  9/30/2024      504               220PROPERTIES LLC   
248

In [ ]:
def clean_loan_data(df):
    # Make a copy to avoid modifying original data
    df_clean = df.copy()

    print("Initial size:", len(df_clean))

    # 1. Clean Loan Status
    if 'Status' in df_clean.columns:
        status_col = 'Status'
    elif 'LoanStatus' in df_clean.columns:
        status_col = 'LoanStatus'

    # Print unique statuses before cleaning
    print("\nUnique loan statuses before cleaning:")
    print(df_clean[status_col].value_counts())

    # Remove canceled loans but keep CHGOFF with zero charge-off
    canceled_mask = df_clean[status_col].str.contains('CANCLD|CANCEL|Canceled', na=False, case=False)
    df_clean = df_clean[~canceled_mask]

    # 2. Check monetary values
    monetary_cols = ['GrossApproval', 'ThirdPartyDollars', 'GrossChargeOffAmount']
    for col in monetary_cols:
        if col in df_clean.columns:
            # Remove negative values
            df_clean = df_clean[df_clean[col].fillna(0) >= 0]

    # 3. Check dates
    date_cols = ['AsOfDate', 'ApprovalDate', 'FirstDisbursementDate',
                'PaidInFullDate', 'ChargeOffDate']

    for col in date_cols:
        if col in df_clean.columns:
            # Convert to datetime
            df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

            # Remove future dates (except AsOfDate which might be a reporting date)
            if col != 'AsOfDate':
                df_clean = df_clean[
                    (df_clean[col].isna()) |
                    (df_clean[col] <= pd.Timestamp.now())
                ]

    # 4. Check logical consistency
    # PaidInFullDate should be after ApprovalDate
    if 'PaidInFullDate' in df_clean.columns and 'ApprovalDate' in df_clean.columns:
        df_clean = df_clean[
            ~((df_clean['PaidInFullDate'].notna()) &
              (df_clean['PaidInFullDate'] < df_clean['ApprovalDate']))
        ]

    # 5. Remove rows with missing crucial information
    crucial_cols = ['BorrName', 'GrossApproval', status_col]
    df_clean = df_clean.dropna(subset=crucial_cols)

    print("\nFinal size after cleaning:", len(df_clean))
    print("\nUnique loan statuses after cleaning:")
    print(df_clean[status_col].value_counts())

    return df_clean

# Clean both datasets
df_10_present_clean = clean_loan_data(df_10_present)
df_91_09_clean = clean_loan_data(df_91_09)

Initial size: 106754

Unique loan statuses before cleaning:
LoanStatus
EXEMPT        53794
PIF           31190
CANCLD        12697
NOT FUNDED     8226
CHGOFF          845
Name: count, dtype: int64

Final size after cleaning: 94055

Unique loan statuses after cleaning:
LoanStatus
EXEMPT        53794
PIF           31190
NOT FUNDED     8226
CHGOFF          845
Name: count, dtype: int64
Initial size: 111341

Unique loan statuses before cleaning:
LoanStatus
PIF       76055
CANCLD    17944
CHGOFF    11233
EXEMPT     5761
Name: count, dtype: int64

Final size after cleaning: 93048

Unique loan statuses after cleaning:
LoanStatus
PIF       76054
CHGOFF    11233
EXEMPT     5761
Name: count, dtype: int64
